# ImkLib Functions
ImgLib2 offers an API to generate virtual functions through the `FunctionRealRandomAccessible` that are evaluated at real locations on demand.

In [1]:
// set up dependencies

// use local repo for now; not deployed to remote maven repo yet
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("net.imglib2:imklib2:0.1.0-SNAPSHOT")
@file:DependsOn("net.imglib2:imglib2-label-multisets:0.8.1")
@file:DependsOn("sc.fiji:bigdataviewer-vistools:1.0.0-beta-21")
%use lets-plot

In [2]:
// for hsv to rgb
import java.awt.Color

import bdv.util.BdvFunctions
import bdv.util.BdvOptions
import kotlin.math.min
import kotlin.math.pow
import kotlin.math.sqrt
import kotlin.random.Random
import net.imglib2.converter.Converter
import net.imglib2.type.numeric.ARGBType
import net.imglib2.type.numeric.integer.IntType
import net.imglib2.imklib.*

In [3]:
val euclideanNormSquared2D = imklib.function(2, { imklib.types.double }) { s, t ->
    t.setReal(s.getDoublePosition(0).pow(2) + s.getDoublePosition(1).pow(2))
}
val interval1 = intArrayOf(-256, -256, 0, 0).intervalMinMax
val interval2 = intArrayOf(0, 0, +256, +256).intervalMinMax
val bdv = BdvFunctions.show(euclideanNormSquared2D.rastered[interval1], "Euclidean norm 2D 1", BdvOptions.options().is2D())
BdvFunctions.show(euclideanNormSquared2D.rastered[interval2], "Euclidean norm 2D 2", BdvOptions.options().is2D().addTo(bdv))

bdv.util.BdvStackSource@638a88f6

## Fractals

In [4]:
// utility function to convert into ARGB space, if desired
// modify this for cool looking results
fun makeConverter(n: Int, random: Random? = null): Converter<IntType, ARGBType> {
    val colors = IntArray(n) {
        val hue = random?.nextDouble(1.0) ?: it.toDouble() / n
        val saturation = 0.3
        val value = it.toDouble() / n
        val rgb = Color.HSBtoRGB(hue.toFloat(), saturation.toFloat(), value.toFloat())
        val r = (rgb shr 16) and 0xff
        val g = (rgb shr  8) and 0xff
        val b = (rgb shr  0) and 0xff
        val argb = ARGBType.rgba(r, g, b, 0)
        if (it == n - 1) 0 else argb
    }
    return Converter { s, t -> t.set(colors[min(s.integer, n - 1)]) }
}

### Julia

Wikipedia has a list of cool values for c: https://en.wikipedia.org/wiki/Julia_set#Quadratic_polynomials

In [5]:
// c&p from Stephan Saalfeld's Java code snippet
// Julia with c = 0.2 + i0.6
fun julia(real: Double, imaginary: Double, r: Double, cRe: Double, cIm: Double, iterations: Int): Int {
    var i = 0
    var re = real
    var im = imaginary
    while (i < iterations && re * re + im * im < r*r) {
        val e = re * re - im * im
        im = 2 * re * im + cIm
        re = e + cRe
        ++i
    }
    return i
}

#### Fixed c and radius

In [6]:
val iterations = 256
val cIm = 0.6
val cRe = 0.2
// play around with r; for r ~ 1, very "interesting looking results"
val r = 2.0
val fractal = imklib.function(2, { imklib.types.int }) { s, t ->
    t.setInteger(julia(s.getDoublePosition(0), s.getDoublePosition(1), r, cRe, cIm, iterations))
}
val interval = intArrayOf(-1, -1, +1, +1).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeConverter(iterations, Random(1)))
// show either convertedFractal or fractal
BdvFunctions.show(convertedFractal, interval, "julia", BdvOptions.options().is2D()).setDisplayRange(0.0, 64.0)

#### Fixed c and variable radius

In [7]:
val iterations = 256
val cIm = 0.6
val cRe = -0.4
val fractal = imklib.function(3, { imklib.types.int }) { s, t ->
    t.setInteger(julia(s.getDoublePosition(0), s.getDoublePosition(1), s.getDoublePosition(2), cRe, cIm, iterations))
}
val interval = intArrayOf(-1, -1, 0, +1, +1, +4).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeConverter(iterations, Random(1)))
// show either convertedFractal or fractal
BdvFunctions.show(convertedFractal, interval, "julia", BdvOptions.options()).setDisplayRange(0.0, 64.0)

### Mandelbrot

In [8]:
fun mandelbrot(real: Double, imaginary: Double, r: Double, iterations: Int): Int {
    var i = 0
    var re = 0.0
    var im = 0.0
    while (i < iterations && re * re + im * im < r*r) {
        val e = re * re - im * im + real
        im = 2 * re * im + imaginary
        re = e
        ++i
    }
    return i
}

In [9]:
val iterations = 256
val fractal = imklib.function(3, { imklib.types.int }) { s, t ->
    t.setInteger(mandelbrot(s.getDoublePosition(0), s.getDoublePosition(1), s.getDoublePosition(2), iterations))
}
val interval = intArrayOf(-1, -1, 0, +1, +1, +4).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeConverter(iterations, Random(1)))
// show either convertedFractal or fractal
BdvFunctions.show(convertedFractal, interval, "mandelbrot", BdvOptions.options()).setDisplayRange(0.0, iterations.toDouble())

### Tricorn

In [10]:
fun tricorn(real: Double, imaginary: Double, r: Double, iterations: Int): Int {
    var i = 0
    var re = 0.0
    var im = 0.0
    while (i < iterations && re * re + im * im < r*r) {
        val e = re * re - im * im + real
        im = -2 * re * im + imaginary
        re = e
        ++i
    }
    return i
}

In [11]:
val iterations = 256
val fractal = imklib.function(3, { imklib.types.int }) { s, t ->
    t.setInteger(tricorn(s.getDoublePosition(0), s.getDoublePosition(1), s.getDoublePosition(2), iterations))
}
val interval = intArrayOf(-1, -1, 0, +1, +1, +4).intervalMinMax
val convertedFractal = fractal.convert(ARGBType(), makeConverter(iterations, Random(1)))
// show either convertedFractal or fractal
BdvFunctions.show(convertedFractal, interval, "tricorn", BdvOptions.options()).setDisplayRange(0.0, iterations.toDouble())